# Movie Ranker

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = (SparkSession.builder
    .master('local')
    .appName('myAppName')
    .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/05 23:31:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read the parquet file generated with pandas and omdb api

In [10]:
df = spark.read.parquet('data/movies.parquet.gzip')
print(f'{df.count()} movies loaded')

243 movies loaded


## filter the movies with a rating less than 6.0

In [11]:
bad_movies = df.filter(col('imdbrating') < 6.0)
bad_movies_count = bad_movies.count()
print(f'We have {bad_movies_count} movies with a ranking less than 6.0')

We have 77 movies with a ranking less than 6.0


In [12]:
bad_movies.sort(col('imdbrating').asc()).show(truncate=False)

+-----------------------------------+----+---------+-----------+-------+----------------------------------+---------+----------+-----------+
|title                              |year|rated    |released   |runtime|genre                             |metascore|imdbrating|boxoffice  |
+-----------------------------------+----+---------+-----------+-------+----------------------------------+---------+----------+-----------+
|Jeepers Creepers: Reborn           |2022|R        |19 Sep 2022|88 min |Horror, Mystery, Thriller         |N/A      |2.5       |$2,033,057 |
|Out of Death                       |2021|R        |16 Jul 2021|96 min |Crime, Thriller                   |N/A      |3.2       |N/A        |
|Warlock III: The End of Innocence  |1999|R        |12 Oct 1999|94 min |Fantasy, Horror, Mystery, Thriller|N/A      |3.8       |N/A        |
|The Visitor                        |2022|N/A      |07 Oct 2022|89 min |Drama, Horror, Mystery            |N/A      |4.0       |N/A        |
|Dark Summer 

In [13]:
unrated_movies = df.where(col('imdbrating') == 'N/A')
print(f'We have {unrated_movies.count()} movies without ranking')

We have 4 movies without ranking


In [14]:
unrated_movies.show(truncate=False)

+--------------------------+----+-----+-----------+-------+-------------------------+---------+----------+---------+
|title                     |year|rated|released   |runtime|genre                    |metascore|imdbrating|boxoffice|
+--------------------------+----+-----+-----------+-------+-------------------------+---------+----------+---------+
|Hellraiser                |2022|R    |07 Oct 2022|121 min|Horror, Mystery, Thriller|56       |N/A       |N/A      |
|The Inhabitant            |2022|N/A  |07 Oct 2022|97 min |Horror, Thriller         |N/A      |N/A       |N/A      |
|Old Man                   |2022|N/A  |14 Oct 2022|97 min |Thriller                 |N/A      |N/A       |N/A      |
|Padre no hay más que uno 3|2022|N/A  |14 Jul 2022|99 min |Comedy                   |N/A      |N/A       |N/A      |
+--------------------------+----+-----+-----------+-------+-------------------------+---------+----------+---------+

